In [ ]:
#Creates initial condition file with temperature and salinities, Coordinate file, and thickness file.
%reset
import numpy as np
import scipy.io as sio
import netCDF4 as nc
import matplotlib.pyplot as plt
%pwd

In [ ]:
#SPECIFY THESE:
XN =400 #NUMBER OF POINTS IN X
YN = 500  #NUMBER OF POINTS IN Y
ZN = 10 #NUMBER OF LAYERS IN Z that are initially filled 
perturbation = np.expand_dims(np.random.random([ZN,YN,XN])*.000001,axis=0)

smat = nc.Dataset('smat.nc')['smat'][:]
tmat = nc.Dataset('tmat.nc')['tmat'][:]
zmat = np.linspace(0,2500,len(smat)) #maximum depth is 2500m

s_grid=np.linspace(34.46,34.914,ZN) #defining salinity points
depth=np.interp(s_grid,smat,zmat) #finding new depth locations for salinity points
depthnew=np.zeros(ZN+1); depthnew[0:ZN-1]=depth[0:ZN-1]; depthnew[ZN-1]=1300; depthnew[ZN]=2500

Sint=np.interp(depthnew,zmat,smat); S_layer=(Sint[1:] + Sint[:-1]) / 2.0;
s_input=np.expand_dims(np.expand_dims(np.expand_dims(S_layer,axis=0),axis=2),axis=3)
s_input=np.repeat(s_input,YN,axis=2)
s_input=np.repeat(s_input,XN,axis=3)+perturbation

Tint=np.interp(depthnew,zmat,tmat); T_layer=(Tint[1:] + Tint[:-1]) / 2.0; 
t_input=np.expand_dims(np.expand_dims(np.expand_dims(T_layer,axis=0),axis=2),axis=3)
t_input=np.repeat(t_input,YN,axis=2)
t_input=np.repeat(t_input,XN,axis=3)+perturbation

h=depthnew[1:ZN+1]-depthnew[0:ZN]
h_input=np.expand_dims(np.expand_dims(np.expand_dims(h,axis=0),axis=2),axis=3)
h_input=np.repeat(h_input,YN,axis=2)
h_input=np.repeat(h_input,XN,axis=3)

eta=-depthnew
eta_input=np.expand_dims(np.expand_dims(np.expand_dims(eta,axis=0),axis=2),axis=3)
eta_input=np.repeat(eta_input,YN,axis=2)
eta_input=np.repeat(eta_input,XN,axis=3)


YC = np.linspace(0.0+100.0/(YN*2),100.0-100.0/(YN*2),YN) #100km in y
XC = np.linspace(0.0+80.0/(XN*2),80.0-80.0/(XN*2),XN) #80km in x



In [ ]:
#EXECUTE THIS CELL TO ADD INITIALLY UNFILLED LAYERS WITH HIGHER SALINITIES, LOWER TEMPERATURES
s_input_adjust=np.zeros((1,ZN+110,YN,XN))
s_input_adjust[0,0:ZN,:,:]=s_input
t_input_adjust=np.zeros((1,ZN+110,YN,XN))
t_input_adjust[0,0:ZN,:,:]=t_input
h_input_adjust=np.zeros((1,ZN+110,YN,XN))
h_input_adjust[0,0:ZN,:,:]=h_input
eta_input_adjust=np.zeros((1,ZN+111,YN,XN))

for i in range(0,XN):
    for j in range(0,YN):
        s_input_adjust[0,10:120,j,i]=np.linspace(34.95,39.45,110)
        t_input_adjust[0,10:120,j,i]=np.linspace(-0.91,-5.0,110) 
        h_input_adjust[0,10:120,j,i]=1.0E-5
eta_input_adjust[0,1:ZN+111,:,:]=-np.cumsum(h_input_adjust,axis=1)

#overwriting previous input variables with expanded/adjusted versions.
ZN=ZN+110
s_input=s_input_adjust
t_input=t_input_adjust
h_input=h_input_adjust
eta_input=eta_input_adjust
eta_input.shape



In [ ]:
fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
img1=plt.plot(s_input[0,:,0,0],linestyle=':',color='b'); plt.title('Salinity coordinates')
plt.xlabel('Layer',fontsize=14)
plt.ylabel('Salinity (psu)',fontsize=14)
plt.show()

fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
img2=plt.plot(t_input[0,:,0,0],linestyle=':',color='b'); plt.title('Temperature coordinates')
plt.xlabel('Layer',fontsize=14)
plt.ylabel('Temperature (C)',fontsize=14)
plt.show()

In [ ]:
print(depthnew[1]-depthnew[0])
print(depthnew[2]-depthnew[1])
print(depthnew[3]-depthnew[2])
print(eta_input[0,:,0,0])
print(np.sum(h_input[0,:,0,0]))


In [ ]:
X, Z = np.meshgrid(XC, -eta_input[0,:,0,0])
fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
img1=plt.pcolor(X, Z, s_input[0,:,0,:],vmin=34.5,vmax=34.9)
for i in range(1,ZN+1,1):
    plt.plot(X[0,:],-eta_input[0,i,0,:],'k',Linewidth=0.2)
plt.gca().invert_yaxis()
plt.title('Salinity', fontsize=14)
plt.xlabel('X Position (km)',fontsize=14)
plt.ylabel('Depth (m)',fontsize=14)
plt.colorbar(img1)
plt.show()

fig=plt.figure(figsize=(7, 4), dpi= 80, facecolor='w', edgecolor='k')
img2=plt.pcolor(X, Z, t_input[0,:,0,:],vmin=-0.89,vmax=0.006)
for i in range(1,ZN+1,1):
    plt.plot(X[0,:],-eta_input[0,i,0,:],'k',Linewidth=0.2)
plt.gca().invert_yaxis()

plt.title('Temperature', fontsize=14)
plt.xlabel('X Position (km)',fontsize=14)
plt.ylabel('Depth (m)',fontsize=14)
plt.colorbar(img2)
plt.show()
print(np.max(s_input))
print(np.min(s_input)) 
print(np.max(t_input))
print(np.min(t_input)) 

In [ ]:
#WRITING INITIAL CONDITIONS
writing = nc.Dataset("IC_TS3D.nc","w",format="NETCDF3_64BIT_OFFSET")
TIME = writing.createDimension("TIME",1)
LAYER = writing.createDimension("LAYER",ZN) 
LAT = writing.createDimension("LAT",YN)
LON = writing.createDimension("LON",XN)


LAT = writing.createVariable("LAT","f8",("LAT"))
LAT[:]=YC
LAT.standard_name = "YC (y coordinate at center of grid cell)"
LAT.units = "km"
LAT.cartesian_axis = "Y"

    
LON = writing.createVariable("LON","f8",("LON"))
LON[:]=XC
LON.standard_name = "XC (x coordinate at center of grid cell)"
LON.units = "km"
LON.cartesian_axis = "X" ;

TIME = writing.createVariable("TIME","f8",("TIME"))
TIME[:]=0.0
TIME.standard_name = "time"
TIME.units = "days since 0001-01-01 00:00:00"
TIME.calendar = "noleap" ;
TIME.cartesian_axis = "T" ;
    

PTEMP = writing.createVariable("PTEMP","f4",("TIME","LAYER","LAT","LON"),fill_value=-1.e+34)
PTEMP[:,:,:,:]=t_input
PTEMP.standard_name = "Initial potential temperature referenced to 0dbar"
PTEMP.units = "Celsius"

SALT = writing.createVariable("SALT","f4",("TIME","LAYER","LAT","LON"),fill_value=-1.e+34)
SALT[:,:,:,:]=s_input
SALT.standard_name = "Initial salinity"
SALT.units = "PSU"


writing.close()

In [ ]:
#NEW FILE CHECK
newfile = nc.Dataset('IC_TS3D.nc')
print(newfile)
temp_new=newfile.variables['PTEMP'][:]
print(temp_new.shape)
salt_new=newfile.variables['SALT'][:]
print(salt_new.shape)

In [ ]:
#WRITING THICKNESS FILE
writing = nc.Dataset("thickness_file.nc","w",format="NETCDF3_64BIT_OFFSET")
TIME = writing.createDimension("TIME",1)
Layer = writing.createDimension("Layer",ZN)
Interface = writing.createDimension("Interface",ZN+1)
LAT = writing.createDimension("LAT",YN)
LON = writing.createDimension("LON",XN)


LAT = writing.createVariable("LAT","f8",("LAT"))
LAT[:]=YC
LAT.standard_name = "YC (y coordinate at center of grid cell)"
LAT.units = "km"
LAT.cartesian_axis = "Y"

    
LON = writing.createVariable("LON","f8",("LON"))
LON[:]=XC
LON.standard_name = "XC (x coordinate at center of grid cell)"
LON.units = "km"
LON.cartesian_axis = "X" ;

TIME = writing.createVariable("TIME","f8",("TIME"))
TIME[:]=0.0
TIME.standard_name = "time"
TIME.units = "days since 0001-01-01 00:00:00"
TIME.calendar = "noleap" ;
TIME.cartesian_axis = "T" ;
    
h = writing.createVariable("h","f4",("TIME","Layer","LAT","LON"),fill_value=-1.e+34)
h[:,:,:,:]=h_input
h.standard_name = "Layer thickness"
h.units = "meters"

eta = writing.createVariable("eta","f4",("TIME","Interface","LAT","LON"),fill_value=-1.e+34)
eta[:,:,:,:]=eta_input
eta.standard_name = "Interface height"
eta.units = "meters"

writing.close()

In [ ]:
#NEW FILE CHECK
newfile = nc.Dataset('thickness_file.nc')
eta_new=newfile.variables['eta'][:]
np.min(eta_new)


In [ ]:
#WRITING COORDINATE FILE OF T AND S
writing = nc.Dataset("Coord_file.nc","w",format="NETCDF3_64BIT_OFFSET")
TIME = writing.createDimension("TIME",1)
DEPTH = writing.createDimension("DEPTH",ZN)

PTEMP = writing.createVariable("PTEMP","f4",("DEPTH"),fill_value=-1.e+34)
PTEMP[:]=t_input[0,:,0,0]
PTEMP.standard_name = "Layer temperature referenced to 0dbar"
PTEMP.units = "Celsius"

SALT = writing.createVariable("SALT","f4",("DEPTH"),fill_value=-1.e+34)
SALT[:]=s_input[0,:,0,0]
SALT.standard_name = "Layer salinity"
SALT.units = "PSU"

writing.close()

In [ ]:
#NEW FILE CHECK
newfile = nc.Dataset('Coord_file.nc')
S_new=newfile.variables['SALT'][:]
T_new=newfile.variables['PTEMP'][:]
np.max(S_new.shape)